In [1]:
import matplotlib.pyplot as plt
import wfdb
import os
import numpy as np
import scipy.signal as signal
import librosa

In [2]:
# database_directory
data_dir = './fetal-ecg-synthetic-database-1.0.0/'
CHUNK = 63 * 768 # 1 second = 44100 fs
NUM_SAMPLES_PER_TRAIN = 5
NUM_SAMPLES_PER_TEST = 1

In [3]:
category = ['c0','c1','c2','c3','c4','c5']

def short_time_fourier_transform(v):
    v = np.asfortranarray(v)
    ft = np.array(librosa.core.stft(v,n_fft=1024,hop_length=768))
    magnitude = np.abs(ft)
    phase = np.angle(ft)
    return magnitude, phase

mix = []
mix_mag = []
mix_pha = []
s1 = []
s1_mag = []
s1_pha = []
s2 = []
s2_mag = []
s2_pha = []

count = 0
for i in os.listdir(data_dir):
    curr_dir = os.path.join(data_dir, i)
    if os.path.isdir(curr_dir) and i[:3] == 'sub' and count < 9:
        count += 1
        print("Working on folder {}:".format(i))
        for j in os.listdir(curr_dir):
            work_dir = os.path.join(curr_dir, j)
            print(" "*4+"Working on {}...".format(j))
            list_folders = os.listdir(work_dir)
            list_folders.sort()
            label = []
            for k in list_folders:
                if k[-4:] == ".dat":
                    split_k = k.split('_')
                    temp_label = '_'.join(split_k[:3])
                    if temp_label not in label:
                        print(" "*8+"processing {}".format(temp_label))
                        label.append(temp_label)
                        for cat in category:
                            fecg = 'fecg1'
                            mecg = 'mecg'
                            noise = 'noise1'
                            fecg_filename = '_'.join([temp_label,cat,fecg])
                            mecg_filename = '_'.join([temp_label,cat,mecg])
                            noise_filename = '_'.join([temp_label,cat,noise])
                            fecg_record = wfdb.rdrecord(os.path.join(work_dir,fecg_filename)).p_signal.transpose(1,0)[12]
                            mecg_record = wfdb.rdrecord(os.path.join(work_dir,mecg_filename)).p_signal.transpose(1,0)[12]
                            noise_record = wfdb.rdrecord(os.path.join(work_dir,noise_filename)).p_signal.transpose(1,0)[12]
                            start_point = np.random.randint(0, high=len(fecg_record)-CHUNK, size=NUM_SAMPLES_PER_TRAIN)
                            mix_record = fecg_record + mecg_record + noise_record
                            for l in start_point:
                                magnitude, phase = short_time_fourier_transform(mix_record[l:l+CHUNK])
                                mix.append(mix_record[l:l+CHUNK])
                                mix_mag.append(magnitude)
                                mix_pha.append(phase)
                                magnitude1, phase = short_time_fourier_transform(mecg_record[l:l+CHUNK])
                                s1.append(mecg_record[l:l+CHUNK])
                                s1_mag.append(magnitude1)
                                s1_pha.append(phase)
                                magnitude2, phase = short_time_fourier_transform(fecg_record[l:l+CHUNK])
                                s2.append(fecg_record[l:l+CHUNK])
                                s2_mag.append(magnitude2)
                                s2_pha.append(phase)
    elif os.path.isdir(curr_dir) and i[:3] == 'sub' and count >= 9:
        # save train data
        subset = 'train'
        np.save('mix_'+subset,np.array(mix))
        np.save('mix_'+subset+'_mag',np.array(mix_mag))
        np.save('mix_'+subset+'_pha',np.array(mix_pha))
        np.save('s1_'+subset,np.array(s1))
        np.save('s1_'+subset+'_mag',np.array(s1_mag))
        np.save('s1_'+subset+'_pha',np.array(s1_pha))
        np.save('s2_'+subset,np.array(s2))
        np.save('s2_'+subset+'_mag',np.array(s2_mag))
        np.save('s2_'+subset+'_pha',np.array(s2_pha))
        subset = 'test'
        mix = []
        mix_mag = []
        mix_pha = []
        s1 = []
        s1_mag = []
        s1_pha = []
        s2 = []
        s2_mag = []
        s2_pha = []
        print("Working on folder {}:".format(i))
        for j in os.listdir(curr_dir):
            work_dir = os.path.join(curr_dir, j)
            print(" "*4+"Working on {}...".format(j))
            list_folders = os.listdir(work_dir)
            list_folders.sort()
            label = []
            for k in list_folders:
                if k[-4:] == ".dat":
                    split_k = k.split('_')
                    temp_label = '_'.join(split_k[:3])
                    if temp_label not in label:
                        print(" "*8+"processing {}".format(temp_label))
                        label.append(temp_label)
                        for cat in category:
                            fecg = 'fecg1'
                            mecg = 'mecg'
                            noise = 'noise1'
                            fecg_filename = '_'.join([temp_label,cat,fecg])
                            mecg_filename = '_'.join([temp_label,cat,mecg])
                            noise_filename = '_'.join([temp_label,cat,noise])
                            fecg_record = wfdb.rdrecord(os.path.join(work_dir,fecg_filename)).p_signal.transpose(1,0)[12]
                            mecg_record = wfdb.rdrecord(os.path.join(work_dir,mecg_filename)).p_signal.transpose(1,0)[12]
                            noise_record = wfdb.rdrecord(os.path.join(work_dir,noise_filename)).p_signal.transpose(1,0)[12]
                            start_point = np.random.randint(0, high=len(fecg_record)-CHUNK, size=NUM_SAMPLES_PER_TRAIN)
                            mix_record = fecg_record + mecg_record + noise_record
                            for l in start_point:
                                magnitude, phase = short_time_fourier_transform(mix_record[l:l+CHUNK])
                                mix.append(mix_record[l:l+CHUNK])
                                mix_mag.append(magnitude)
                                mix_pha.append(phase)
                                magnitude1, phase = short_time_fourier_transform(mecg_record[l:l+CHUNK])
                                s1.append(mecg_record[l:l+CHUNK])
                                s1_mag.append(magnitude1)
                                s1_pha.append(phase)
                                magnitude2, phase = short_time_fourier_transform(fecg_record[l:l+CHUNK])
                                s2.append(fecg_record[l:l+CHUNK])
                                s2_mag.append(magnitude2)
                                s2_pha.append(phase)
        np.save('mix_'+subset,np.array(mix))
        np.save('mix_'+subset+'_mag',np.array(mix_mag))
        np.save('mix_'+subset+'_pha',np.array(mix_pha))
        np.save('s1_'+subset,np.array(s1))
        np.save('s1_'+subset+'_mag',np.array(s1_mag))
        np.save('s1_'+subset+'_pha',np.array(s1_pha))
        np.save('s2_'+subset,np.array(s2))
        np.save('s2_'+subset+'_mag',np.array(s2_mag))
        np.save('s2_'+subset+'_pha',np.array(s2_pha))
        break

Working on folder sub10:
    Working on snr00dB...
        processing sub10_snr00dB_l1
        processing sub10_snr00dB_l2
        processing sub10_snr00dB_l3
        processing sub10_snr00dB_l4
        processing sub10_snr00dB_l5
    Working on snr09dB...
        processing sub10_snr09dB_l1
        processing sub10_snr09dB_l2
        processing sub10_snr09dB_l3
        processing sub10_snr09dB_l4
        processing sub10_snr09dB_l5
    Working on snr03dB...
        processing sub10_snr03dB_l1
        processing sub10_snr03dB_l2
        processing sub10_snr03dB_l3
        processing sub10_snr03dB_l4
        processing sub10_snr03dB_l5
    Working on snr06dB...
        processing sub10_snr06dB_l1
        processing sub10_snr06dB_l2
        processing sub10_snr06dB_l3
        processing sub10_snr06dB_l4
        processing sub10_snr06dB_l5
    Working on snr12dB...
        processing sub10_snr12dB_l1
        processing sub10_snr12dB_l2
        processing sub10_snr12dB_l3
        processin

        processing sub01_snr06dB_l5
    Working on snr12dB...
        processing sub01_snr12dB_l1
        processing sub01_snr12dB_l2
        processing sub01_snr12dB_l3
        processing sub01_snr12dB_l4
        processing sub01_snr12dB_l5
Working on folder sub06:
    Working on snr00dB...
        processing sub06_snr00dB_l1
        processing sub06_snr00dB_l2
        processing sub06_snr00dB_l3
        processing sub06_snr00dB_l4
        processing sub06_snr00dB_l5
    Working on snr09dB...
        processing sub06_snr09dB_l1
        processing sub06_snr09dB_l2
        processing sub06_snr09dB_l3
        processing sub06_snr09dB_l4
        processing sub06_snr09dB_l5
    Working on snr03dB...
        processing sub06_snr03dB_l1
        processing sub06_snr03dB_l2
        processing sub06_snr03dB_l3
        processing sub06_snr03dB_l4
        processing sub06_snr03dB_l5
    Working on snr06dB...
        processing sub06_snr06dB_l1
        processing sub06_snr06dB_l2
        processin